In [1]:
import pandas as pd
import joblib

In [2]:
df=pd.read_csv("C://Users//nithin goud//Downloads//mental.csv")
df.head()

,Age,Gender,Region,Industry,Job_Role,Work_Arrangement,Hours_Per_Week,Burnout_Level,Work_Life_Balance_Score,Physical_Health_Issues,Social_Isolation_Score,Salary_Range
0,27,Female,Asia,Professional Services,Data Analyst,Onsite,64,High,3,Shoulder Pain; Neck Pain,2,$40K-60K
1,37,Female,Asia,Professional Services,Data Analyst,Onsite,37,High,4,Back Pain,2,$80K-100K
2,32,Female,Africa,Education,Business Analyst,Onsite,36,High,3,Shoulder Pain; Eye Strain,2,$80K-100K
3,40,Female,Europe,Education,Data Analyst,Onsite,63,Medium,1,Shoulder Pain; Eye Strain,2,$60K-80K
4,30,Male,South America,Manufacturing,DevOps Engineer,Hybrid,65,Medium,5,NaN,4,$60K-80K


In [3]:
df.columns

Index(['Age', 'Gender', 'Region', 'Industry', 'Job_Role', 'Work_Arrangement',
       'Hours_Per_Week', 'Burnout_Level', 'Work_Life_Balance_Score',
       'Physical_Health_Issues', 'Social_Isolation_Score', 'Salary_Range'],
      dtype='object')

In [13]:
model=joblib.load('random_forest.pkl')
burn=joblib.load('burn_map.pkl')
salary=joblib.load('salary_order.pkl')
health=joblib.load('health_pkl')
nom=joblib.load('nominal.pkl')
label=joblib.load('lable.pkl')
nominal_enc=joblib.load("C://Users//nithin goud//Mental Health Project//dummy_columns.pkl")

In [14]:
nominal_enc

['Gender_Male',
 'Gender_Non-binary',
 'Gender_Prefer not to say',
 'Region_Asia',
 'Region_Europe',
 'Region_North America',
 'Region_Oceania',
 'Region_South America',
 'Industry_Education',
 'Industry_Finance',
 'Industry_Healthcare',
 'Industry_Manufacturing',
 'Industry_Marketing',
 'Industry_Professional Services',
 'Industry_Retail',
 'Industry_Technology',
 'Job_Role_Business Analyst',
 'Job_Role_Consultant',
 'Job_Role_Content Writer',
 'Job_Role_Customer Service Manager',
 'Job_Role_Data Analyst',
 'Job_Role_Data Scientist',
 'Job_Role_DevOps Engineer',
 'Job_Role_Digital Marketing Specialist',
 'Job_Role_Executive Assistant',
 'Job_Role_Financial Analyst',
 'Job_Role_HR Manager',
 'Job_Role_IT Support',
 'Job_Role_Marketing Specialist',
 'Job_Role_Operations Manager',
 'Job_Role_Product Manager',
 'Job_Role_Project Manager',
 'Job_Role_Quality Assurance',
 'Job_Role_Research Scientist',
 'Job_Role_Sales Representative',
 'Job_Role_Social Media Manager',
 'Job_Role_Software E

In [ ]:
import pandas as pd
import joblib
from sklearn.preprocessing import MultiLabelBinarizer

# --- Load models & encoders ---
model = joblib.load("C://Users//nithin goud//Mental Health Project//random_forest.pkl")
burn_map = joblib.load("C://Users//nithin goud//Mental Health Project//burn_map.pkl")
salary_enc = joblib.load("C://Users//nithin goud//Mental Health Project//salary_order.pkl")
label_enc = joblib.load("C://Users//nithin goud//Mental Health Project//lable.pkl")
health_enc = joblib.load("C://Users//nithin goud//Mental Health Project//health_pkl")
nominal_enc = joblib.load("C://Users//nithin goud//Mental Health Project//nominal.pkl")

# --- Load data for dropdown references ---
inp = pd.read_csv("C://Users//nithin goud//Downloads//mental.csv")

while True:
    print("\n--- Mental Health Status Prediction ---")
    age = eval(input(f"Enter Age: Range -> {inp.Age.min()} to {inp.Age.max()}: "))
    gender = input(f"Gender {list(inp.Gender.unique())}: ")
    region = input(f"Region {list(inp.Region.unique())}: ")
    industry = input(f"Industry {list(inp.Industry.unique())}: ")
    job_role = input(f"Job Role {list(inp.Job_Role.unique())}: ")
    work_arrangement = input(f"Work Arrangement {list(inp.Work_Arrangement.unique())}: ")
    hours_per_week = int(input("Hours per Week: "))
    burnout_level = input(f"Burnout Level {list(inp.Burnout_Level.unique())}: ")
    work_life_balance = int(input("Work Life Balance (1-5): "))
    health_issues = input(f"Physical Health Issues {list(inp.Physical_Health_Issues.unique())}: ")
    social_isolation_score = int(input("Social Isolation Score (1-5): "))
    salary_range = input(f"Salary Range {list(inp.Salary_Range.unique())}: ")

    # --- Create DataFrame for input ---
    input_df = pd.DataFrame([[
        age, gender, region, industry, job_role, work_arrangement, hours_per_week,
        burnout_level, work_life_balance, health_issues, social_isolation_score, salary_range
    ]], columns=inp.columns)

    # --- Encode burnout ---
    if hasattr(burn_map, 'transform'):
        input_df['Burnout_Level'] = burn_map.transform(input_df[['Burnout_Level']])
    else:
        input_df['Burnout_Level'] = input_df['Burnout_Level'].map(burn_map)

    # --- Encode salary ---
    if hasattr(salary_enc, 'transform'):
        input_df['Salary_Range'] = salary_enc.transform(input_df[['Salary_Range']])
    else:
        input_df['Salary_Range'] = input_df['Salary_Range'].map(salary_enc)

    # --- Encode health issues ---
    input_df['Physical_Health_Issues'] = input_df['Physical_Health_Issues'].apply(
        lambda x: x.split(',') if isinstance(x, str) and x.lower() != 'none' else []
    )
    mlb = MultiLabelBinarizer(classes=list(health_enc))
    mlb.fit([list(health_enc)])  # ensures same order as training
    health_array = mlb.transform(input_df['Physical_Health_Issues'])
    health_df = pd.DataFrame(health_array, columns=mlb.classes_, index=input_df.index)
    input_df = pd.concat([input_df.drop('Physical_Health_Issues', axis=1), health_df], axis=1)

    # --- Encode nominal features (exact same order as training) ---
    nominal_cols_fitted = nominal_enc.feature_names_in_
    nominal_data = input_df[nominal_cols_fitted]
    nominal_encoded = nominal_enc.transform(nominal_data)
    if hasattr(nominal_encoded, "toarray"):
        nominal_encoded = nominal_encoded.toarray()
    nominal_df = pd.DataFrame(
        nominal_encoded,
        columns=nominal_enc.get_feature_names_out(nominal_cols_fitted),
        index=input_df.index
    )

    # Drop original nominal cols and append encoded
    input_df = input_df.drop(columns=nominal_cols_fitted)
    input_df = pd.concat([input_df, nominal_df], axis=1)

    # --- Match model features exactly ---
    for col in model.feature_names_in_:
        if col not in input_df.columns:
            input_df[col] = 0  # add missing with 0
    input_df = input_df[model.feature_names_in_]  # reorder

    # --- Predict ---
    prediction_encoded = model.predict(input_df)
    if hasattr(label_enc, 'inverse_transform'):
        prediction = label_enc.inverse_transform(prediction_encoded)
    else:
        prediction = prediction_encoded

    # --- Predict probabilities ---
    probabilities = model.predict_proba(input_df)[0]
    class_labels = label_enc.inverse_transform(range(len(probabilities)))
    proba_df = pd.DataFrame({"Class": class_labels, "Probability": probabilities})

    print(f"\n✅ Predicted Mental Health Status: {prediction[0]}")
    print("\n📊 Probabilities:")
    print(proba_df.to_string(index=False))




--- Mental Health Status Prediction ---
Enter Age: Range -> 22 to 65: 45
Gender ['Female', 'Male', 'Non-binary', 'Prefer not to say']: Male
Region ['Asia', 'Africa', 'Europe', 'South America', 'Oceania', 'North America']: Asia
Industry ['Professional Services', 'Education', 'Manufacturing', 'Customer Service', 'Technology', 'Finance', 'Retail', 'Healthcare', 'Marketing']: Education
Job Role ['Data Analyst', 'Business Analyst', 'DevOps Engineer', 'IT Support', 'Technical Writer', 'Software Engineer', 'HR Manager', 'Project Manager', 'Data Scientist', 'UX Designer', 'Social Media Manager', 'Digital Marketing Specialist', 'Account Manager', 'Research Scientist', 'Sales Representative', 'Operations Manager', 'Customer Service Manager', 'Product Manager', 'Quality Assurance', 'Executive Assistant', 'Financial Analyst', 'Marketing Specialist', 'Consultant', 'Content Writer']: Consultant
Work Arrangement ['Onsite', 'Hybrid', 'Remote']: Hybrid
Hours per Week: 45
Burnout Level ['High', 'Medium

C:\Users\nithin goud\AppData\Roaming\Python\Python311\site-packages\sklearn\preprocessing\_label.py:909: UserWarning: unknown class(es) ['Black Pain'] will be ignored
  warnings.warn(



✅ Predicted Mental Health Status: adhd

📊 Probabilities:
          Class  Probability
           adhd        0.215
        anxiety        0.170
        burnout        0.145
     depression        0.125
           none        0.170
           ptsd        0.070
stress disorder        0.105

--- Mental Health Status Prediction ---
Enter Age: Range -> 22 to 65: 25
Gender ['Female', 'Male', 'Non-binary', 'Prefer not to say']: Male
Region ['Asia', 'Africa', 'Europe', 'South America', 'Oceania', 'North America']: Asia
Industry ['Professional Services', 'Education', 'Manufacturing', 'Customer Service', 'Technology', 'Finance', 'Retail', 'Healthcare', 'Marketing']: Marketing
Job Role ['Data Analyst', 'Business Analyst', 'DevOps Engineer', 'IT Support', 'Technical Writer', 'Software Engineer', 'HR Manager', 'Project Manager', 'Data Scientist', 'UX Designer', 'Social Media Manager', 'Digital Marketing Specialist', 'Account Manager', 'Research Scientist', 'Sales Representative', 'Operations Manag